#### **Team members**:  
#####           -Godwin Bime, 
#####           -Yin Yin Tan, 
#####           -Pratik Patel, 
#####           -Crystal Li

In [3]:
from pyspark.sql.window import Window
from pyspark.sql import functions as f
from pyspark.sql import Row
#import pandas as pd
#import matplotlib.pyplot as plt
#import seaborn as sns

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
%%info

ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
9,application_1683841041393_0010,pyspark,idle,Link,Link,None,✔


### S3 bucket where the datasets are stored

In [64]:
s3_bucket = 's3://ai620groupprojectbucket/'
prefix = 'AI620'
output_destination_loc = prefix + 'objec2vec/'

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### Install pyspark packages

In [7]:
# sc.install_pypi_package("pandas==1.0.5")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [8]:
# sc.install_pypi_package("matplotlib", "https://pypi.org/simple")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [9]:
# sc.install_pypi_package("pyarrow==1.0.0")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [10]:
# sc.install_pypi_package("seaborn")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### View packages installed

In [11]:
sc.list_packages()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Package                    Version
-------------------------- ----------
aws-cfn-bootstrap          2.0
beautifulsoup4             4.9.3
boto                       2.49.0
click                      8.1.3
docutils                   0.14
jmespath                   1.0.1
joblib                     1.2.0
lockfile                   0.11.0
lxml                       4.9.2
mysqlclient                1.4.2
nltk                       3.8
nose                       1.3.4
numpy                      1.20.0
pip                        20.2.2
py-dateutil                2.2
pystache                   0.5.4
python-daemon              2.2.3
python37-sagemaker-pyspark 1.4.2
pytz                       2022.7
PyYAML                     5.4.1
regex                      2021.11.10
setuptools                 28.8.0
simplejson                 3.2.0
six                        1.13.0
tqdm                       4.64.1
wheel                      0.29.0
windmill                   1.6


### Uninstall pyspark packages

In [12]:
# sc.uninstall_package('pandas')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### Read the tweets dataset

### Read training dataset

In [13]:
train = spark.read.option("header", "true").option("delimeter", ",").csv(s3_bucket + "twitter_training.csv")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### Read testing dataset

In [14]:
test = spark.read.option("header", "true").option("delimeter", ",").csv(s3_bucket + "twitter_validation.csv")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### Dataset attribute names

In [15]:
column_names = ['Tweet_ID', 'Entity', 'Sentiment', 'Tweet']

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### Assign column/attribute names

In [16]:
train=train.toDF(*column_names)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [17]:
test=test.toDF(*column_names)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [18]:
train.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

74681

In [19]:
test.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

1509

In [20]:
train.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+-----------+---------+--------------------+
|Tweet_ID|     Entity|Sentiment|               Tweet|
+--------+-----------+---------+--------------------+
|    2401|Borderlands| Positive|I am coming to th...|
|    2401|Borderlands| Positive|im getting on bor...|
|    2401|Borderlands| Positive|im coming on bord...|
|    2401|Borderlands| Positive|im getting on bor...|
|    2401|Borderlands| Positive|im getting into b...|
+--------+-----------+---------+--------------------+
only showing top 5 rows

In [43]:
test.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+---------+---------+--------------------+
|Tweet_ID|   Entity|Sentiment|               Tweet|
+--------+---------+---------+--------------------+
|     352|   Amazon|  Neutral|BBC News - Amazon...|
|    8312|Microsoft| Negative|@Microsoft Why do...|
|    4371|    CS-GO| Negative|CSGO matchmaking ...|
|    4433|   Google|  Neutral|Now the President...|
|    6273|     FIFA| Negative|Hi @EAHelp I’ve h...|
+--------+---------+---------+--------------------+
only showing top 5 rows

### Obtain sentiment and tweet across all tweet IDs

In [21]:
train.select("Sentiment", "Tweet").show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------+--------------------+
|Sentiment|               Tweet|
+---------+--------------------+
| Positive|I am coming to th...|
| Positive|im getting on bor...|
| Positive|im coming on bord...|
| Positive|im getting on bor...|
| Positive|im getting into b...|
+---------+--------------------+
only showing top 5 rows

In [44]:
test.select("Sentiment", "Tweet").show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------+--------------------+
|Sentiment|               Tweet|
+---------+--------------------+
|  Neutral|BBC News - Amazon...|
| Negative|@Microsoft Why do...|
| Negative|CSGO matchmaking ...|
|  Neutral|Now the President...|
| Negative|Hi @EAHelp I’ve h...|
+---------+--------------------+
only showing top 5 rows

### Describe the datasets

In [22]:
train.describe().show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+------------------+---------------+----------+--------------------+
|summary|          Tweet_ID|         Entity| Sentiment|               Tweet|
+-------+------------------+---------------+----------+--------------------+
|  count|             74681|          74681|     74681|               73995|
|   mean|   6432.6401494356|           null|      null|                 3.2|
| stddev|3740.4238192995094|           null|      null|   2.007130147392398|
|    min|                 1|         Amazon|Irrelevant|                    |
|    max|              9999|johnson&johnson|  Positive|🧻 at Home Depot ...|
+-------+------------------+---------------+----------+--------------------+

In [45]:
test.describe().show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+--------------------+-----------------+--------------------+--------------------+
|summary|            Tweet_ID|           Entity|           Sentiment|               Tweet|
+-------+--------------------+-----------------+--------------------+--------------------+
|  count|                1509|             1042|                1014|                 999|
|   mean|    6435.15915915916|             null|                null|                null|
| stddev|   3728.912225900553|             null|                null|                null|
|    min|            100 V...| 2020 at 11:31AM"| ...  and free at...|"""A delayed game...|
|    max|     🥉@cristianhs30|  johnson&johnson|            Positive|🤣👉 @KEEMSTAR Ya...|
+-------+--------------------+-----------------+--------------------+--------------------+

### Check for null values in the datasets and drop them

In [49]:
# train.na.drop().show()
train.na.fill('Neutral').show(7)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+-----------+---------+--------------------+
|Tweet_ID|     Entity|Sentiment|               Tweet|
+--------+-----------+---------+--------------------+
|    2401|Borderlands| Positive|I am coming to th...|
|    2401|Borderlands| Positive|im getting on bor...|
|    2401|Borderlands| Positive|im coming on bord...|
|    2401|Borderlands| Positive|im getting on bor...|
|    2401|Borderlands| Positive|im getting into b...|
|    2402|Borderlands| Positive|So I spent a few ...|
|    2402|Borderlands| Positive|So I spent a coup...|
+--------+-----------+---------+--------------------+
only showing top 7 rows

In [50]:
# test.na.drop().show()
test.na.fill('Neutral').show(7)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+---------+---------+--------------------+
|            Tweet_ID|   Entity|Sentiment|               Tweet|
+--------------------+---------+---------+--------------------+
|                 352|   Amazon|  Neutral|BBC News - Amazon...|
|                8312|Microsoft| Negative|@Microsoft Why do...|
|                4371|    CS-GO| Negative|CSGO matchmaking ...|
|                4433|   Google|  Neutral|Now the President...|
|                6273|     FIFA| Negative|Hi @EAHelp I’ve h...|
|                7925|MaddenNFL| Positive|Thank you @EAMadd...|
|New TE Austin Hoo...|  Neutral|  Neutral|             Neutral|
+--------------------+---------+---------+--------------------+
only showing top 7 rows

In [51]:
### Datadecription after removing null values

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [52]:
train.describe().show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+------------------+---------------+----------+--------------------+
|summary|          Tweet_ID|         Entity| Sentiment|               Tweet|
+-------+------------------+---------------+----------+--------------------+
|  count|             74681|          74681|     74681|               73995|
|   mean|   6432.6401494356|           null|      null|                 3.2|
| stddev|3740.4238192995094|           null|      null|   2.007130147392398|
|    min|                 1|         Amazon|Irrelevant|                    |
|    max|              9999|johnson&johnson|  Positive|🧻 at Home Depot ...|
+-------+------------------+---------------+----------+--------------------+

### Group the dataset by Sentiment

In [53]:
train.groupBy('Sentiment').count().show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+-----+
| Sentiment|count|
+----------+-----+
|Irrelevant|12990|
|  Positive|20831|
|   Neutral|18318|
|  Negative|22542|
+----------+-----+

### Count and display a unique ID of single column

In [54]:
train.select('Tweet_ID').distinct().show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+
|Tweet_ID|
+--------+
|     296|
|   13192|
|   12529|
|    1090|
|    7252|
|    5325|
|    1512|
|    1572|
|   11888|
|   10436|
|    9630|
|    9830|
|     451|
|    8306|
|    1394|
|    6366|
|    2700|
|    1870|
|    2917|
|    7248|
+--------+
only showing top 20 rows

In [55]:
train.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- Tweet_ID: string (nullable = true)
 |-- Entity: string (nullable = true)
 |-- Sentiment: string (nullable = true)
 |-- Tweet: string (nullable = true)

In [56]:
train.columns

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['Tweet_ID', 'Entity', 'Sentiment', 'Tweet']

### Read the first tweet

In [57]:
train.first()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Row(Tweet_ID='2401', Entity='Borderlands', Sentiment='Positive', Tweet='I am coming to the borders and I will kill you all,')

### Convert sql dataframe to Pandas then use it for data visualization

In [58]:
# train = train.toPandas()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [59]:
# sns.histplot(train['Tweet'].str.len(), binwidth=50)
# plt.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [60]:
# train.info()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### Count information per category

In [61]:
# data_count=train.groupby(by=["Entity","Sentiment"]).count().reset_index()
# data_count.head()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### Save the prepared dataset in parquet format

In [65]:
train.write.parquet(s3_bucket + output_destination_loc + "/traintweetsentiments.paquet", mode='overwrite')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [66]:
test.write.parquet(s3_bucket + output_destination_loc + "/testtweetsentiments.paquet", mode='overwrite')


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…